In [ ]:
!pip install -q datasets==2.21.0 requests torch peft bitsandbytes transformers trl accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.6.1 which is incompatible.


In [ ]:
import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed
from peft import LoraConfig, PeftModel
from datetime import datetime

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from huggingface_hub import HfApi, create_repo, upload_file
import os

# Load JSON dataset
json_file = "/content/shree_ravi_shankar_data.json"  # Replace with your JSON file path
df = pd.read_json(json_file)

# Split into train and test sets
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Save to Parquet format
train_file = "train.parquet"
test_file = "test.parquet"
train_df.to_parquet(train_file, index=False)
test_df.to_parquet(test_file, index=False)

# Hugging Face Hub details
repo_name = "sri-sri-ravishankar-chat-style"  # Replace with your repo name
api = HfApi()

# Authenticate and fetch username
user = api.whoami()["name"]
repo_id = f"{user}/{repo_name}"

# Create the repository if it doesn't exist
try:
    create_repo(repo_id, repo_type="dataset", exist_ok=True)
    print(f"Repository created or already exists: https://huggingface.co/datasets/{repo_id}")
except Exception as e:
    print(f"Error creating repository: {e}")
    raise

# Upload files to the repository
for file, file_path in [("train.parquet", train_file), ("test.parquet", test_file)]:
    try:
        upload_file(
            path_or_fileobj=file_path,
            path_in_repo=file,
            repo_id=repo_id,
            repo_type="dataset",
        )
        print(f"Uploaded {file} to https://huggingface.co/datasets/{repo_id}/{file}")
    except Exception as e:
        print(f"Error uploading {file}: {e}")

print(f"Dataset successfully uploaded to: https://huggingface.co/datasets/{repo_id}")


Repository created or already exists: https://huggingface.co/datasets/grudgie/sri-sri-ravishankar-chat-style


train.parquet:   0%|          | 0.00/142k [00:00<?, ?B/s]

Uploaded train.parquet to https://huggingface.co/datasets/grudgie/sri-sri-ravishankar-chat-style/train.parquet


test.parquet:   0%|          | 0.00/59.0k [00:00<?, ?B/s]

Uploaded test.parquet to https://huggingface.co/datasets/grudgie/sri-sri-ravishankar-chat-style/test.parquet
Dataset successfully uploaded to: https://huggingface.co/datasets/grudgie/sri-sri-ravishankar-chat-style


In [ ]:
BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
PROJECT_NAME = "Ravi-shankar-chat-style"
HF_USER = "grudgie"  # your HF name here!

# Data
DATASET_NAME = f"grudgie/sri-sri-ravishankar-chat-style"
MAX_SEQUENCE_LENGTH = 182

# Run name for saving the model in the hub
RUN_NAME = f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}_{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"

# Hyperparameters for LoRA
LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]
LORA_DROPOUT = 0.1
QUANT_4_BIT = True

# Hyperparameters for Training
EPOCHS = 3
BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 1
LEARNING_RATE = 1e-4
LR_SCHEDULER_TYPE = "cosine"
WARMUP_RATIO = 0.03
OPTIMIZER = "paged_adamw_32bit"

# Admin config
STEPS = 50
SAVE_STEPS = 500
LOG_TO_WANDB = True

%matplotlib inline

In [ ]:
# Log in to Hugging Face
from huggingface_hub import login
from wandb import wandb
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

# Log in to Weights & Biases
wandb_api_key = userdata.get('WANDB_API_KEY')
os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.login()

# Configure Weights & Biases to record against our project
os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoints"
os.environ["WANDB_WATCH"] = "gradients"


wandb: Currently logged in as: srivastava-dhruv78 (ds-iiitu). Use `wandb login --relogin` to force relogin


In [ ]:
from datasets import load_dataset
dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
if LOG_TO_WANDB:
    wandb.init(project=PROJECT_NAME, name=RUN_NAME)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
if QUANT_4_BIT:
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4"
    )
else:
    quant_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_compute_dtype=torch.bfloat16
    )

In [ ]:
# Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(f"Memory footprint: {(base_model.get_memory_footprint()/1e6):.1f} MB")

In [ ]:
from trl import SFTConfig, SFTTrainer

lora_parameters = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=TARGET_MODULES,
)

# Next, specify the general configuration parameters for training
train_parameters = SFTConfig(
    output_dir=PROJECT_RUN_NAME,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=1,
    eval_strategy="no",
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    optim=OPTIMIZER,
    save_steps=SAVE_STEPS,
    save_total_limit=10,
    logging_steps=STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=WARMUP_RATIO,
    group_by_length=True,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    report_to="wandb" if LOG_TO_WANDB else None,
    run_name=RUN_NAME,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    dataset_text_field="text",
    save_strategy="steps",
    hub_strategy="every_save",
    push_to_hub=True,
    hub_model_id=HUB_MODEL_NAME,
    hub_private_repo=True
)

In [ ]:
# And now, the Supervised Fine Tuning Trainer will carry out the fine-tuning
# Given these 2 sets of configuration parameters

fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=new_train,
    eval_dataset=new_test,
    peft_config=lora_parameters,
    tokenizer=tokenizer,
    args=train_parameters,
    data_collator=collator
)

In [ ]:
# Fine-tune!
fine_tuning.train()

# Push our fine-tuned model to Hugging Face
fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

if LOG_TO_WANDB:
    wandb.finish()

## Prediction

In [1]:
!pip install -q datasets==2.21.0 requests torch peft bitsandbytes transformers trl accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# imports

import os
import re

import math
from tqdm import tqdm

from google.colab import userdata
from huggingface_hub import login

import torch
import torch.nn.functional as F

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, set_seed

from datasets import load_dataset, Dataset, DatasetDict
from datetime import datetime

from peft import PeftModel
import matplotlib.pyplot as plt

In [4]:
# Constants

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"

HF_USER = "grudgie" # your HF name here! Or use mine if you just want to reproduce my resul

# The run itself

RUN_NAME = "Ravi-shankar-chat-style_2025-01-31_04.29.51"

REVISION = None
FINETUNED_MODEL = f"grudgie/{RUN_NAME}"

# Data

DATASET_NAME = f"{HF_USER}/sri-sri-ravishankar-chat-style"

# Hyperparameters for QLoRA

QUANT_4_BIT = True




In [5]:
from huggingface_hub import login
hf_token = userdata.get("HF_TOKEN")
login(hf_token)

In [6]:
from datasets import load_dataset
dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']

Generating train split:   0%|          | 0/646 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/216 [00:00<?, ? examples/s]

In [7]:
test[0]

{'question': 'Today is my birthday. I want this day to be the beginning of a more spiritual life and therefore something has to be done. Please tell me what I must do.',
 'answer': 'It is not with the mind that one should decide what has to be done. It should be a spontaneous movement taking place in a sincere and constant aspiration.'}

In [8]:
if QUANT_4_BIT:
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4"
    )
else:
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_compute_dtype=torch.bfloat16
    )


In [9]:

# Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

base_model.generation_config.pad_token_id = tokenizer.pad_token_id

# Load the fine-tuned model with PEFT

if REVISION:
    fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL, revision=REVISION)
else:
    fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/751 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

In [30]:

# Original prediction function takes the most likely next token

def model_predict(prompt):
  set_seed(42)

  inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
  attention_mask = torch.ones(inputs.shape, device="cuda")

  outputs = fine_tuned_model.generate(inputs, attention_mask=attention_mask,temperature=0.0001,repetition_penalty=1.5, max_new_tokens=200, num_return_sequences=1)
  response = tokenizer.decode(outputs[0])

  return response

In [27]:
test[0]['question']

'Today is my birthday. I want this day to be the beginning of a more spiritual life and therefore something has to be done. Please tell me what I must do.'

In [28]:
model_predict(test[0]['question'])

'<|begin_of_text|>Today is my birthday. I want this day to be the beginning of a more spiritual life and therefore something has to be done. Please tell me what I must do. What should one wish for on their birthdays? How can they make it an important moment in order that from then onwards, things may change?\nThe first thing you have told us shows clearly how much we are attached here below; if our body does not feel well or there’s some trouble with friends… But no! We don’t come into existence just like that – each time someone comes down he brings his own karma which makes him suffer certain illnesses at such-and-such moments because these were part of its past lives’ experiences too (and so forth). The second point concerns desires: when people ask themselves “What shall happen tomorrow?” instead asking quietly within oneself whether indeed anything will occur differently than usual without any particular reason being given beforehand as though by chance alone could bring about cha

In [31]:

# Original prediction function takes the most likely next token

def model_predict(prompt):
  set_seed(42)

  inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
  attention_mask = torch.ones(inputs.shape, device="cuda")

  outputs = fine_tuned_model.generate(inputs, attention_mask=attention_mask,temperature=1,repetition_penalty=1.5, max_new_tokens=200, num_return_sequences=1)
  response = tokenizer.decode(outputs[0])

  return response

In [32]:
test[1]['question']

'What is the remedy for understanding our being?'

In [33]:
model_predict(test[1]['question'])

'<|begin_of_text|>What is the remedy for understanding our being? What method must we follow to attain this kind of knowledge?\nThe first thing you have got to do, I told him (a disciple), and it’s something very important; well! The main business with oneself—and if one isn’t able at least from time to time in twenty-four hours—I insist on that—for ten minutes even—if not every day—to be alone by yourself: now when sitting there before your soul without looking out—there are a thousand things which will draw off all attention—it may happen while travelling. For example someone comes along—a friend or somebody else—you begin talking so much about stupidities—or more serious affairs sometimes—one gets involved just like…like as they say “caught up”—and afterwards what happens most often—at times almost always—is precisely either falling asleep because people bore each other stiff after half an hour talkin’ gossipy nonsense—that tires everybody immediately then especially during summer h